# Importacion de librerias

In [1]:
from pyspark.sql import SparkSession


## Creando Sesion de spark con nombre Practica

In [3]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

## Leer el archivo adult.csv sin encabezado e infiriendo el esquema 

In [4]:
df_pyspark = spark.read.option('header','false').csv('adult.csv',inferSchema=True)

In [5]:
df_pyspark.show(15)

+---+----------------+------+------------+---+--------------------+-----------------+-------------+------------------+------+-----+----+----+-------------+-----+
|_c0|             _c1|   _c2|         _c3|_c4|                 _c5|              _c6|          _c7|               _c8|   _c9| _c10|_c11|_c12|         _c13| _c14|
+---+----------------+------+------------+---+--------------------+-----------------+-------------+------------------+------+-----+----+----+-------------+-----+
| 39|       State-gov| 77516|   Bachelors| 13|       Never-married|     Adm-clerical|Not-in-family|             White|  Male| 2174|   0|  40|United-States|<=50K|
| 50|Self-emp-not-inc| 83311|   Bachelors| 13|  Married-civ-spouse|  Exec-managerial|      Husband|             White|  Male|    0|   0|  13|United-States|<=50K|
| 38|         Private|215646|     HS-grad|  9|            Divorced|Handlers-cleaners|Not-in-family|             White|  Male|    0|   0|  40|United-States|<=50K|
| 53|         Private|234721

## Definiendo un encabezado para el df

In [6]:
df_pyspark= df_pyspark.toDF(*("age","workclass","fnlwgt","education","educationNum","maritalStatus","occupation","relationship","race","sex","capitalGain","capitalLoss","hoursPerWeek","nativeCountry","Mayor"))

## Imprimir el esquema inferido

In [7]:
df_pyspark.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educationNum: integer (nullable = true)
 |-- maritalStatus: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capitalGain: integer (nullable = true)
 |-- capitalLoss: integer (nullable = true)
 |-- hoursPerWeek: integer (nullable = true)
 |-- nativeCountry: string (nullable = true)
 |-- Mayor: string (nullable = true)



In [8]:
df_pyspark.show()

+---+----------------+------+------------+------------+--------------------+-----------------+-------------+------------------+------+-----------+-----------+------------+-------------+-----+
|age|       workclass|fnlwgt|   education|educationNum|       maritalStatus|       occupation| relationship|              race|   sex|capitalGain|capitalLoss|hoursPerWeek|nativeCountry|Mayor|
+---+----------------+------+------------+------------+--------------------+-----------------+-------------+------------------+------+-----------+-----------+------------+-------------+-----+
| 39|       State-gov| 77516|   Bachelors|          13|       Never-married|     Adm-clerical|Not-in-family|             White|  Male|       2174|          0|          40|United-States|<=50K|
| 50|Self-emp-not-inc| 83311|   Bachelors|          13|  Married-civ-spouse|  Exec-managerial|      Husband|             White|  Male|          0|          0|          13|United-States|<=50K|
| 38|         Private|215646|     HS-gra

## Crear un df sin ningun dato nulo 

In [12]:
df_pyspark_DropNA_any= df_pyspark.na.drop(how='any')

In [13]:
df_pyspark_DropNA_any.show()

+---+----------------+------+------------+------------+--------------------+-----------------+-------------+------------------+------+-----------+-----------+------------+-------------+-----+
|age|       workclass|fnlwgt|   education|educationNum|       maritalStatus|       occupation| relationship|              race|   sex|capitalGain|capitalLoss|hoursPerWeek|nativeCountry|Mayor|
+---+----------------+------+------------+------------+--------------------+-----------------+-------------+------------------+------+-----------+-----------+------------+-------------+-----+
| 39|       State-gov| 77516|   Bachelors|          13|       Never-married|     Adm-clerical|Not-in-family|             White|  Male|       2174|          0|          40|United-States|<=50K|
| 50|Self-emp-not-inc| 83311|   Bachelors|          13|  Married-civ-spouse|  Exec-managerial|      Husband|             White|  Male|          0|          0|          13|United-States|<=50K|
| 38|         Private|215646|     HS-gra

## Contar total de datos en df y df sin datos nulos

In [15]:
df_pyspark.count()

32561

In [16]:
df_pyspark_DropNA_any.count()

30162

## Importacion de tras librerias 


In [17]:
from pyspark.sql.functions import col,isnan, when, count

## Definir una consulta para contar el total de datos nulos

In [18]:
b = df_pyspark.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df_pyspark.columns])
b.show()

+---+---------+------+---------+------------+-------------+----------+------------+----+---+-----------+-----------+------------+-------------+-----+
|age|workclass|fnlwgt|education|educationNum|maritalStatus|occupation|relationship|race|sex|capitalGain|capitalLoss|hoursPerWeek|nativeCountry|Mayor|
+---+---------+------+---------+------------+-------------+----------+------------+----+---+-----------+-----------+------------+-------------+-----+
|  0|     1836|     0|        0|           0|            0|      1843|           0|   0|  0|          0|          0|           0|          583|    0|
+---+---------+------+---------+------------+-------------+----------+------------+----+---+-----------+-----------+------------+-------------+-----+



## Transformar los datos nulos de la columna occupation por el dato "No occupation"

In [29]:
df_pyspark_NoNull= df_pyspark.na.fill("No occupation",['occupation'])

In [48]:
df_pyspark_NoNull.show(100)

+---+----------------+------+------------+------------+--------------------+-----------------+--------------+------------------+------+-----------+-----------+------------+-----------------+-----+
|age|       workclass|fnlwgt|   education|educationNum|       maritalStatus|       occupation|  relationship|              race|   sex|capitalGain|capitalLoss|hoursPerWeek|    nativeCountry|Mayor|
+---+----------------+------+------------+------------+--------------------+-----------------+--------------+------------------+------+-----------+-----------+------------+-----------------+-----+
| 39|       State-gov| 77516|   Bachelors|          13|       Never-married|     Adm-clerical| Not-in-family|             White|  Male|       2174|          0|          40|    United-States|<=50K|
| 50|Self-emp-not-inc| 83311|   Bachelors|          13|  Married-civ-spouse|  Exec-managerial|       Husband|             White|  Male|          0|          0|          13|    United-States|<=50K|
| 38|         P

## Transformar los datos nulos de la columna nativeCountry por el dato "No country  native"

In [31]:
df_pyspark_NoNull= df_pyspark_NoNull.na.fill("No country Native",['nativeCountry'])

In [49]:
df_pyspark_NoNull.show(100)

+---+----------------+------+------------+------------+--------------------+-----------------+--------------+------------------+------+-----------+-----------+------------+-----------------+-----+
|age|       workclass|fnlwgt|   education|educationNum|       maritalStatus|       occupation|  relationship|              race|   sex|capitalGain|capitalLoss|hoursPerWeek|    nativeCountry|Mayor|
+---+----------------+------+------------+------------+--------------------+-----------------+--------------+------------------+------+-----------+-----------+------------+-----------------+-----+
| 39|       State-gov| 77516|   Bachelors|          13|       Never-married|     Adm-clerical| Not-in-family|             White|  Male|       2174|          0|          40|    United-States|<=50K|
| 50|Self-emp-not-inc| 83311|   Bachelors|          13|  Married-civ-spouse|  Exec-managerial|       Husband|             White|  Male|          0|          0|          13|    United-States|<=50K|
| 38|         P

## Transformar los datos nulos de la columna workclass por el dato "No workclass"

In [34]:
df_pyspark_NoNull = df_pyspark_NoNull.na.fill("No WorkClass",['workclass'])

In [50]:
df_pyspark_NoNull.show(100)

+---+----------------+------+------------+------------+--------------------+-----------------+--------------+------------------+------+-----------+-----------+------------+-----------------+-----+
|age|       workclass|fnlwgt|   education|educationNum|       maritalStatus|       occupation|  relationship|              race|   sex|capitalGain|capitalLoss|hoursPerWeek|    nativeCountry|Mayor|
+---+----------------+------+------------+------------+--------------------+-----------------+--------------+------------------+------+-----------+-----------+------------+-----------------+-----+
| 39|       State-gov| 77516|   Bachelors|          13|       Never-married|     Adm-clerical| Not-in-family|             White|  Male|       2174|          0|          40|    United-States|<=50K|
| 50|Self-emp-not-inc| 83311|   Bachelors|          13|  Married-civ-spouse|  Exec-managerial|       Husband|             White|  Male|          0|          0|          13|    United-States|<=50K|
| 38|         P

## Comparacion de valores nulos con nuevo consulta en el df sin datos nulos

In [36]:
b = df_pyspark.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df_pyspark.columns])
b.show()

+---+---------+------+---------+------------+-------------+----------+------------+----+---+-----------+-----------+------------+-------------+-----+
|age|workclass|fnlwgt|education|educationNum|maritalStatus|occupation|relationship|race|sex|capitalGain|capitalLoss|hoursPerWeek|nativeCountry|Mayor|
+---+---------+------+---------+------------+-------------+----------+------------+----+---+-----------+-----------+------------+-------------+-----+
|  0|     1836|     0|        0|           0|            0|      1843|           0|   0|  0|          0|          0|           0|          583|    0|
+---+---------+------+---------+------------+-------------+----------+------------+----+---+-----------+-----------+------------+-------------+-----+



In [37]:
J = df_pyspark_NoNull.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df_pyspark_NoNull.columns])
J.show()

+---+---------+------+---------+------------+-------------+----------+------------+----+---+-----------+-----------+------------+-------------+-----+
|age|workclass|fnlwgt|education|educationNum|maritalStatus|occupation|relationship|race|sex|capitalGain|capitalLoss|hoursPerWeek|nativeCountry|Mayor|
+---+---------+------+---------+------------+-------------+----------+------------+----+---+-----------+-----------+------------+-------------+-----+
|  0|        0|     0|        0|           0|            0|         0|           0|   0|  0|          0|          0|           0|            0|    0|
+---+---------+------+---------+------------+-------------+----------+------------+----+---+-----------+-----------+------------+-------------+-----+



In [57]:
df_pyspark_NoNull.show(15)

+---+----------------+------+------------+------------+--------------------+-----------------+-------------+------------------+------+-----------+-----------+------------+-----------------+-----+
|age|       workclass|fnlwgt|   education|educationNum|       maritalStatus|       occupation| relationship|              race|   sex|capitalGain|capitalLoss|hoursPerWeek|    nativeCountry|Mayor|
+---+----------------+------+------------+------------+--------------------+-----------------+-------------+------------------+------+-----------+-----------+------------+-----------------+-----+
| 39|       State-gov| 77516|   Bachelors|          13|       Never-married|     Adm-clerical|Not-in-family|             White|  Male|       2174|          0|          40|    United-States|<=50K|
| 50|Self-emp-not-inc| 83311|   Bachelors|          13|  Married-civ-spouse|  Exec-managerial|      Husband|             White|  Male|          0|          0|          13|    United-States|<=50K|
| 38|         Privat

In [56]:
df_pyspark.show(15)

+---+----------------+------+------------+------------+--------------------+-----------------+-------------+------------------+------+-----------+-----------+------------+-------------+-----+
|age|       workclass|fnlwgt|   education|educationNum|       maritalStatus|       occupation| relationship|              race|   sex|capitalGain|capitalLoss|hoursPerWeek|nativeCountry|Mayor|
+---+----------------+------+------------+------------+--------------------+-----------------+-------------+------------------+------+-----------+-----------+------------+-------------+-----+
| 39|       State-gov| 77516|   Bachelors|          13|       Never-married|     Adm-clerical|Not-in-family|             White|  Male|       2174|          0|          40|United-States|<=50K|
| 50|Self-emp-not-inc| 83311|   Bachelors|          13|  Married-civ-spouse|  Exec-managerial|      Husband|             White|  Male|          0|          0|          13|United-States|<=50K|
| 38|         Private|215646|     HS-gra

In [41]:
df_pyspark_DropNA_any2= df_pyspark_NoNull.na.drop(how='any')

In [58]:
df_pyspark_DropNA_any.count()

30162

In [46]:
df_pyspark_DropNA_any2.count()

32561

In [47]:
df_pyspark_NoNull.count()

32561